### Import libraries 

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, PrintfTickFormatter, Title, Text, LabelSet

output_notebook()

Loading BokehJS ...

### Our objetive 

<img src="../../images/figure_10.png" width="50%" height="50%">

#### Preprocessing 

In [2]:
dataset = pd.read_csv('../../data/data_director_population.csv')
dataset.shape

(6, 6)

In [3]:
dataset.head()

,Date,Attrition,Today's directors,Promotions to director,Directors from acquisitions,Unmet need
0,Today,NaN,191,NaN,NaN,NaN
1,FY16,-26.0,165,7.0,3.0,28.0
2,FY17,-43.0,148,10.0,3.0,50.0
3,FY18,-62.0,129,13.0,3.0,68.0
4,FY19,-80.0,111,17.0,3.0,91.0


In [4]:
dataset_pos = dataset.drop(columns=['Attrition']).copy()

In [5]:
dataset_vbar_pos = dataset_pos.set_index('Date').T.rename_axis('Type').reset_index()

In [6]:
dataset_vbar_pos.columns.name = None

In [7]:
dataset_vbar_pos

,Type,Today,FY16,FY17,FY18,FY19,FY20
0,Today's directors,191.0,165.0,148.0,129.0,111.0,91.0
1,Promotions to director,NaN,7.0,10.0,13.0,17.0,21.0
2,Directors from acquisitions,NaN,3.0,3.0,3.0,3.0,3.0
3,Unmet need,NaN,28.0,50.0,68.0,91.0,112.0


In [8]:
dataset_neg = dataset[['Date', 'Attrition']].copy()

In [9]:
dataset_neg

,Date,Attrition
0,Today,NaN
1,FY16,-26.0
2,FY17,-43.0
3,FY18,-62.0
4,FY19,-80.0
5,FY20,-100.0


In [10]:
dataset_vbar_neg = dataset_neg.set_index('Date').T.rename_axis('Type').reset_index()

In [11]:
dataset_vbar_neg

Date,Type,Today,FY16,FY17,FY18,FY19,FY20
0,Attrition,NaN,-26.0,-43.0,-62.0,-80.0,-100.0


In [12]:
# Bokeh doesn't allow you to add off-axis text. This is a little trick to do it
# This code adds extra space on the right
new_row = {'Date':''}
dataset = dataset.append(new_row, ignore_index=True)

new_row = {'Date':' '}
dataset = dataset.append(new_row, ignore_index=True)

new_row = {'Date':'  '}
dataset = dataset.append(new_row, ignore_index=True)

#### Colors 

In [13]:
GRAY1, GRAY2, GRAY3 = '#231F20', '#414040', '#555655'
GRAY4, GRAY5, GRAY6 = '#646369', '#76787B', '#828282'
GRAY7, GRAY8, GRAY9 = '#929497', '#A6A6A5', '#BFBEBE'
BLUE1, BLUE2, BLUE3, BLUE4 = '#174A7E', '#4A81BF', '#94B2D7', '#94AFC5'
RED1, RED2 = '#C3514E', '#E6BAB7'
GREEN1, GREEN2 = '#0C8040', '#9ABB59'
ORANGE1 = '#F79747'

#### Font 

In [14]:
FONT = 'Arial'

In [15]:
dataset_pos['position'] = dataset_pos.set_index('Date').cumsum(axis=1).reset_index()['Unmet need']

In [16]:
dataset_pos

,Date,Today's directors,Promotions to director,Directors from acquisitions,Unmet need,position
0,Today,191,NaN,NaN,NaN,NaN
1,FY16,165,7.0,3.0,28.0,203.0
2,FY17,148,10.0,3.0,50.0,211.0
3,FY18,129,13.0,3.0,68.0,213.0
4,FY19,111,17.0,3.0,91.0,222.0
5,FY20,91,21.0,3.0,112.0,227.0


#### Plot 

In [31]:
# Set the source of the plot
source_pos = ColumnDataSource(dataset_pos)
source_neg = ColumnDataSource(dataset_neg)

segments = dataset['Date'].to_list()

customer_pos = dataset_vbar_pos['Type'].to_list()
customer_neg = dataset_vbar_neg['Type'].to_list()

# Set color by category to the stacked bar
colors = [BLUE2, GREEN1, GREEN2, 'WHITE']


# Create the figure
p = figure(x_range=segments,
           y_range=(-120, 275),
           plot_height=670,
           plot_width=850, 
           title='Expected director population over time',
           toolbar_location='above')


# Add bars to the figure
p.vbar_stack(customer_pos, 
             x='Date',
             color=colors,
             width=0.7, 
             line_color=GRAY6,
             line_width=0.7, 
             source=source_pos)

p.add_layout(LabelSet(x='Date', 
                      y='position', 
                      x_offset=0, 
                      y_offset=-5, 
                      text='Unmet need',                      
                      text_align='center',
                      text_baseline='top', 
                      text_color=GRAY2,
                      text_font=FONT,
                      text_font_size='16pt',
                      text_font_style='bold',
                      source=source_pos))

# Add bars to the figure
p.vbar_stack(customer_neg, 
             x='Date',
             color=BLUE3,
             width=0.7, 
             line_color='white',
             line_width=0.7, 
             source=source_neg)

p.add_layout(LabelSet(x='Date', 
                      y=-5, 
                      x_offset=0, 
                      y_offset=-5, 
                      text='Date',
                      text_align='center',
                      text_baseline='top', 
                      text_color=GRAY4,
                      text_font=FONT,
                      text_font_size='14pt',
                      source=source_neg))


# Add footing legend

p.add_layout(Title(text='A footnote explaining relevant forecast assumptions and methodology would go here.', 
                   align="left",
                   offset=-40,
                   text_color=GRAY8,
                   text_font=FONT,
                   text_font_size='11pt',
                   text_font_style='normal'), 
             "below")

# Before annotations
p.text(x=0, 
       y=-35, 
       x_offset=14,
       text=['9/30/15'],       
       text_color=GRAY4, 
       text_font=FONT,
       text_font_size='14pt')

# Before annotations
p.text(x=6, 
       y=227, 
       x_offset=-5,
       y_offset=26,
       text=['Unmet need (gap)'],       
       text_color=GRAY2, 
       text_font=FONT,
       text_font_size='17pt',
       text_font_style='bold')

p.text(x=6, 
       y=114, 
       x_offset=-5,
       y_offset=10,
       text=['Directors from acquisitions'],       
       text_color=GREEN2, 
       text_font=FONT,
       text_font_size='13pt')

p.text(x=6, 
       y=100, 
       x_offset=-5,
       y_offset=10,
       text=['Promotions to director'],       
       text_color=GREEN1, 
       text_font=FONT,
       text_font_size='13pt')

p.text(x=6, 
       y=86, 
       x_offset=-5,
       y_offset=10,
       text=['Todays directors'],       
       text_color=BLUE2, 
       text_font=FONT,
       text_font_size='13pt')

p.text(x=6, 
       y=-100, 
       x_offset=-5,
       y_offset=5,
       text=['Attrition'],       
       text_color=BLUE4, 
       text_font=FONT,
       text_font_size='13pt')

# Elements attributes

# Modify title attributes
p.title.offset = -65
p.title.text_color = GRAY4
p.title.text_font = FONT
p.title.text_font_size = '22pt'
p.title.text_font_style = 'normal'

p.xaxis.bounds = (0, 6)
p.xaxis.fixed_location = 0
p.xaxis.axis_line_color = GRAY8
p.xaxis.major_label_text_color = None
p.xaxis.major_tick_line_color = None
p.xaxis.minor_tick_line_color = None
p.xgrid.grid_line_color = None

p.yaxis.bounds = (-100, 250)
p.yaxis.axis_line_color = GRAY9
p.yaxis.axis_label = ' ' * 100 + '# of directors'             
p.yaxis.axis_label_standoff = 5  
p.yaxis.axis_label_text_color = GRAY8
p.yaxis.axis_label_text_font = FONT
p.yaxis.axis_label_text_font_size = '12pt'
p.yaxis.axis_label_text_font_style = 'normal' 
p.yaxis.major_label_standoff = 5
p.yaxis.major_label_text_color = GRAY7
p.yaxis.major_label_text_font = FONT
p.yaxis.major_label_text_font_size = '12pt'
p.yaxis.major_tick_in = 0
p.yaxis.major_tick_out = 3
p.yaxis.major_tick_line_color = GRAY9
p.yaxis.minor_tick_line_color = None
p.ygrid.grid_line_color = None

# Convert the figure to png
p.background_fill_color = None
p.border_fill_color = None
p.outline_line_color = None

show(p)